In [5]:
#INTODUCTION TO COLLABORATIVE FILTERING 
#Here we are having a matrix which shows the ratings given by a praticular user for various movies. Of course all users would not give ratings for all the movies. So with the help of 
#collaborative filtering we are trying to predict the rating which the user might have given to the movies he hasnt rate yet.Here
#the ratings of other users are also taken into consideration. Hence the name collaborative


#importing the required libraries

#we are usign the dataser form ratings_small.csv which is samller verison of the original dataset. We cann however use the original data set as well 
#but takes a very long time

import pandas as pd
import surprise
ratings= pd.read_csv("ratings_small.csv")

In [6]:
#we have used the surpise library which takes care of the Single Value Decompostion
from surprise import Reader
from surprise import Dataset
#from surprise import evaluate
from surprise import SVD


In [7]:
#discarding the timestamp column which is very useless
ratings=ratings.drop(['timestamp'],axis=1)

In [8]:
from surprise.model_selection import cross_validate

from surprise.model_selection import GridSearchCV


reader=Reader()
data = Dataset.load_from_df(ratings,reader)
#using GridSearch from Suprise library

#here number of epochs,learning rate,regularization Parameter.

#we are trying to optimise our algorithm by hypertuning our parameters

param_grid = {
    "n_epochs": [5, 10, 15],
    "lr_all": [0.001,0.002, 0.003],
    "reg_all": [0.01, 0.02, 0.03]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3,n_jobs=-1)
gs.fit(data)
a = gs.best_params["rmse"]
print(gs.best_score["rmse"])

0.9076661835652069


In [9]:
reader=Reader()
algo = SVD(n_epochs = a['n_epochs'] ,lr_all =  a['lr_all'], reg_all = a['reg_all'])



In [10]:
#performing cross validation (using 4 folds)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=4, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 4 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Mean    Std     
RMSE (testset)    0.9092  0.9044  0.9095  0.8951  0.9046  0.0058  
MAE (testset)     0.7022  0.6977  0.7032  0.6926  0.6989  0.0042  
Fit time          3.06    3.09    3.07    3.07    3.07    0.01    
Test time         0.20    0.22    0.21    0.22    0.21    0.01    


{'test_rmse': array([0.90921276, 0.9044303 , 0.90951276, 0.89513909]),
 'test_mae': array([0.7021506 , 0.69772128, 0.70316369, 0.69256751]),
 'fit_time': (3.060966968536377,
  3.0895118713378906,
  3.071460008621216,
  3.0747227668762207),
 'test_time': (0.2030940055847168,
  0.21695518493652344,
  0.21354174613952637,
  0.21617889404296875)}

In [11]:
#here we need entire data to be trained so we take an inbuilt function called build_full_trainset
training_data = data.build_full_trainset()
algo.fit(training_data)


In [12]:
#We can predict how much ratings the user may give for a perticular movie.
algo.predict(1,580)

Prediction(uid=1, iid=580, r_ui=None, est=2.8731420369225784, details={'was_impossible': False})